In [ ]:
pip install nltk

In [14]:
pip install newspaper3k

In [15]:
pip install googletrans==3.1.0a0

In [16]:
from newspaper import Article
import random
import string
import numpy as np
import warnings
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from googletrans import Translator

In [17]:
warnings.filterwarnings('ignore')

In [18]:
nltk.download('punkt',quiet=True)


True

In [19]:
article = Article("https://en.wikipedia.org/wiki/Coronavirus_disease_2019")
article.download()
article.parse()
article.nlp()
corpus=article.text
print(corpus)

Disease caused by severe acute respiratory syndrome coronavirus 2

"COVID" redirects here. For the ongoing pandemic, see COVID-19 pandemic . For other diseases caused by coronaviruses, see Coronavirus diseases

Medical condition

Coronavirus disease 2019 (COVID-19), also known as the coronavirus, or COVID, is a contagious disease caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). The first known case was identified in Wuhan, China, in December 2019.[7] The disease has since spread worldwide, leading to an ongoing pandemic.[8]

Symptoms of COVID-19 are variable, but often include fever,[9] cough, headache,[10] fatigue, breathing difficulties, and loss of smell and taste.[11][12] Symptoms may begin one to fourteen days after exposure to the virus. At least a third of people who are infected do not develop noticeable symptoms.[13] Of those people who develop noticeable symptoms enough to be classed as patients, most (81%) develop mild to moderate symptoms (up to mild 

In [20]:
text=corpus
sent_tokens=nltk.sent_tokenize(text)
print(sent_tokens)

['Disease caused by severe acute respiratory syndrome coronavirus 2\n\n"COVID" redirects here.', 'For the ongoing pandemic, see COVID-19 pandemic .', 'For other diseases caused by coronaviruses, see Coronavirus diseases\n\nMedical condition\n\nCoronavirus disease 2019 (COVID-19), also known as the coronavirus, or COVID, is a contagious disease caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2).', 'The first known case was identified in Wuhan, China, in December 2019.', '[7] The disease has since spread worldwide, leading to an ongoing pandemic.', '[8]\n\nSymptoms of COVID-19 are variable, but often include fever,[9] cough, headache,[10] fatigue, breathing difficulties, and loss of smell and taste.', '[11][12] Symptoms may begin one to fourteen days after exposure to the virus.', 'At least a third of people who are infected do not develop noticeable symptoms.', '[13] Of those people who develop noticeable symptoms enough to be classed as patients, most (81%) develop 

In [21]:
remove_punct_dict=dict( (ord(punct),None) for punct in string.punctuation)
print(string.punctuation)
print(remove_punct_dict)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
{33: None, 34: None, 35: None, 36: None, 37: None, 38: None, 39: None, 40: None, 41: None, 42: None, 43: None, 44: None, 45: None, 46: None, 47: None, 58: None, 59: None, 60: None, 61: None, 62: None, 63: None, 64: None, 91: None, 92: None, 93: None, 94: None, 95: None, 96: None, 123: None, 124: None, 125: None, 126: None}


In [22]:
def LemNormalize(text):
  return nltk.word_tokenize(text.lower().translate(remove_punct_dict))
print(LemNormalize(text))

['disease', 'caused', 'by', 'severe', 'acute', 'respiratory', 'syndrome', 'coronavirus', '2', 'covid', 'redirects', 'here', 'for', 'the', 'ongoing', 'pandemic', 'see', 'covid19', 'pandemic', 'for', 'other', 'diseases', 'caused', 'by', 'coronaviruses', 'see', 'coronavirus', 'diseases', 'medical', 'condition', 'coronavirus', 'disease', '2019', 'covid19', 'also', 'known', 'as', 'the', 'coronavirus', 'or', 'covid', 'is', 'a', 'contagious', 'disease', 'caused', 'by', 'severe', 'acute', 'respiratory', 'syndrome', 'coronavirus', '2', 'sarscov2', 'the', 'first', 'known', 'case', 'was', 'identified', 'in', 'wuhan', 'china', 'in', 'december', '20197', 'the', 'disease', 'has', 'since', 'spread', 'worldwide', 'leading', 'to', 'an', 'ongoing', 'pandemic8', 'symptoms', 'of', 'covid19', 'are', 'variable', 'but', 'often', 'include', 'fever9', 'cough', 'headache10', 'fatigue', 'breathing', 'difficulties', 'and', 'loss', 'of', 'smell', 'and', 'taste1112', 'symptoms', 'may', 'begin', 'one', 'to', 'fourte

In [23]:
def response(user_response):
 #user response and robo responce
  #user_response="WHat is chronic disease"
  translater = Translator()
  translated_response = translater.translate(user_response, dest= "en")
  out = translated_response.text
  user_response = out.lower()
  #print(user_response)

    #print(user_response)
  #robo response
  robo_response=''
  sent_tokens.append(user_response)
  #print(sent_tokens)
  tfidfvec=TfidfVectorizer(tokenizer=LemNormalize , stop_words='english')
  tfidf=tfidfvec.fit_transform(sent_tokens)
    
    #get similarity score
  val=cosine_similarity(tfidf[-1],tfidf)
  #print(val)
  idx=val.argsort()[0][-2]
  flat=val.flatten()
  flat.sort()
  score=flat[-2]
  #print(score)
  if score==0:
    robo_response=robo_response+"sorry,i dont understand"
    
  else:
    robo_response=robo_response+sent_tokens[idx]

  translated_robo_response = translater.translate(robo_response, dest= "hi")  
  sent_tokens.remove(user_response)
  return (translated_robo_response.text) 

In [ ]:
greeting_input=["नमस्ते","हैलो"]
greeting_response=["नमस्ते","हैलो","आपका स्वागत है","सुस्वागतम :)"]
def greeting(sentence):
  for word in sentence.split():
    if word.lower() in greeting_input:
      return random.choice(greeting_response)
flag=True
print("Doc: नमस्ते!!! यह डॉक्टर बॉट है, मैं कोविड-19 से संबंधित आपके प्रश्नों का उत्तर दे सकता हूं, बाहर निकलने के लिए अलविदा टाइप करें")
while(flag==True):
  user_response=input("You:")
  #user_response=user_response.lower()
  if(user_response!='अलविदा'):
    if(user_response=='धन्यवाद' or user_response=='शुक्रिया'):
      flag=False
      print("Doc: मदद करने में खुशी हुई")
    else:
       if( greeting(user_response) != None):
         print("Doc: "+ greeting(user_response))
       else:
         print("Doc:"+ response(user_response))
  else:
    flag=False
    print("Doc: फिर मिलेंगे :)")


Doc: नमस्ते!!! यह डॉक्टर बॉट है, मैं कोविड-19 से संबंधित आपके प्रश्नों का उत्तर दे सकता हूं, बाहर निकलने के लिए अलविदा टाइप करें
You:कोविड-19 का पहला ज्ञात मामला
Doc:पहले ज्ञात मामले की पहचान दिसंबर 2019 में चीन के वुहान में हुई थी।
You:जनता के लिए क्या सिफारिश की गई है
Doc:प्रसारण के जोखिम को कम करने के लिए सार्वजनिक सेटिंग्स में फेस मास्क या कवरिंग के उपयोग की सिफारिश की गई है।
You:कोविड -19 से बचने के उपय
Doc:महामारी की रोकथाम के उपायों के बिना - जैसे कि सामाजिक दूरी, टीकाकरण और फेस मास्क - रोगजनक तेजी से फैल सकते हैं। यह ग्राफिक दिखाता है कि रोकथाम के उपायों को जल्दी अपनाने से आबादी के व्यापक क्षेत्रों की रक्षा होती है।
You:स्टेरॉयड का कोविड में क्या असर होता है?
Doc:[४०२] क्योंकि यह एक अच्छी तरह से परीक्षण किया गया और व्यापक रूप से उपलब्ध उपचार है, डब्ल्यूएचओ द्वारा इसका स्वागत किया गया, जो डेक्सामेथासोन और अन्य स्टेरॉयड को शामिल करने के लिए उपचार दिशानिर्देशों को अद्यतन करने की प्रक्रिया में है।
